In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import json
from datetime import datetime, timedelta, date # Ensure 'date' is imported
import pytz
import traceback # For more detailed error reporting

# --- Configuration ---
# Set the start date for processing
start_date_str = '20251010'  # Replace with your desired start date in YYYYMMDD format

# The ArcGIS Online Item ID of the Feature Service
item_id = "6a371d1a491a4a0794578b031859c768"

# Output directory for GeoJSON files
output_directory = "F:/Data/Las_Vegas_Crime/Data/LVMPD/LVMPD_Calls_For_Service/FileTypes/GeoJSON/Daily/"
output_file_prefix = "LVMPD_Calls_For_Service_"

# --- Main Script ---
try:
    # 1. Initialize dates
    start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
    # Use .date() to ensure we are comparing dates, not datetimes, for the loop
    actual_today = datetime.now(pytz.utc).date()

    print(f"Script started. Processing from {start_date.strftime('%Y-%m-%d')} to {actual_today.strftime('%Y-%m-%d')}.")

    # 2. Connect to GIS and get the Feature Layer
    # Using anonymous access. If authentication is needed, configure GIS() accordingly.
    # Example: gis = GIS("your_arcgis_online_url", "username", "password")
    gis = GIS()
    item = gis.content.get(item_id)

    if item is None:
        print(f"Error: Item with ID '{item_id}' not found.")
    elif item.type == "Feature Service":
        if not item.layers:
            print(f"Error: Feature Service with ID '{item_id}' has no layers.")
        else:
            layer = item.layers[0] # Assuming the data is in the first layer
            feature_layer_object = FeatureLayer(layer.url, gis=gis)

            # 3. Download all features from the layer ONCE
            # This query fetches all features. If the dataset is extremely large,
            # this might be slow or consume a lot of memory.
            # The fields "Day" and "Year" are assumed to exist in the feature attributes
            # for local filtering, based on the original script's where_clause.
            print(f"Attempting to query all features from layer: {layer.properties.name}...")
            try:
                all_features_set = feature_layer_object.query(
                    where="1=1",  # Fetches all features
                    as_df=False,
                    out_sr=4326,      # Output spatial reference WGS84
                    return_geometry=True
                )
                print(f"Successfully queried {len(all_features_set.features)} features in total.")
            except Exception as e:
                print(f"Error during initial feature layer query: {e}")
                raise # Re-raise to stop execution if data download fails

            # 4. Loop through each day from start_date to actual_today
            current_date_in_loop = start_date
            while current_date_in_loop <= actual_today:
                target_year = current_date_in_loop.year
                target_day_of_year = current_date_in_loop.timetuple().tm_yday

                print(f"\nProcessing data for: {current_date_in_loop.strftime('%Y-%m-%d')} (Year: {target_year}, DayOfYear: {target_day_of_year})")

                # 5. Filter features for the current_date_in_loop from the downloaded set
                daily_filtered_features = []
                for feature in all_features_set.features:
                    attributes = feature.attributes
                    # Ensure 'Day' and 'Year' attributes exist and are numeric or can be cast to int
                    feature_day_attr = attributes.get("Day")
                    feature_year_attr = attributes.get("Year")

                    if feature_day_attr is not None and feature_year_attr is not None:
                        try:
                            if int(feature_day_attr) == target_day_of_year and int(feature_year_attr) == target_year:
                                daily_filtered_features.append(feature)
                        except ValueError:
                            # Log if 'Day' or 'Year' cannot be converted to int
                            # print(f"Warning: Non-integer Day/Year for OBJECTID {attributes.get('OBJECTID')}: Day='{feature_day_attr}', Year='{feature_year_attr}'")
                            pass # Or handle as an error
                    # else:
                        # print(f"Warning: Feature OBJECTID {attributes.get('OBJECTID')} missing Day or Year attribute for filtering.")


                if not daily_filtered_features:
                    print(f"No features found for {current_date_in_loop.strftime('%Y-%m-%d')}.")
                    current_date_in_loop += timedelta(days=1)
                    continue

                # 6. Convert filtered features to GeoJSON format for the current day
                try:
                    geojson_features_for_day = []
                    for feature in daily_filtered_features:
                        attributes = feature.attributes
                        geometry = feature.geometry # This is a dictionary for Point geometries

                        geojson_feature = {
                            "type": "Feature",
                            "id": attributes.get("OBJECTID"), # Assuming OBJECTID is the unique ID
                            "geometry": {
                                "type": "Point", # Assuming Point geometry as per original
                                "coordinates": [geometry['x'], geometry['y']] if geometry and 'x' in geometry and 'y' in geometry else None
                            },
                            "properties": attributes.copy() # Use a copy to avoid modifying the original dict
                        }

                        # Format date fields within properties
                        for key in ["IncidentDate", "UpdatedDate"]:
                            if geojson_feature["properties"].get(key):
                                timestamp_ms = geojson_feature["properties"][key]
                                if isinstance(timestamp_ms, (int, float)): # Check if it's a number
                                    try:
                                        dt_object = datetime.fromtimestamp(timestamp_ms / 1000, tz=pytz.utc)
                                        geojson_feature["properties"][key] = dt_object.strftime("%a, %d %b %Y %H:%M:%S GMT")
                                    except Exception as ts_ex:
                                        print(f"Warning: Could not parse timestamp for {key} in OBJECTID {attributes.get('OBJECTID')}: {timestamp_ms}. Error: {ts_ex}")
                                        # Decide what to do: keep original, set to None, or remove
                                        # geojson_feature["properties"][key] = None
                                else:
                                    # print(f"Warning: Timestamp for {key} in OBJECTID {attributes.get('OBJECTID')} is not a number: {timestamp_ms}")
                                    pass # Keep original non-numeric value or handle as needed

                        geojson_features_for_day.append(geojson_feature)

                    geojson_data_for_day = {
                        "type": "FeatureCollection",
                        "crs": {
                            "type": "name",
                            "properties": {"name": "EPSG:4326"}
                        },
                        "features": geojson_features_for_day
                    }

                    # 7. Save the GeoJSON data to a file
                    output_file_name = f"{output_directory}{output_file_prefix}{current_date_in_loop.strftime('%Y%m%d')}.geojson"
                    
                    # Ensure output directory exists (optional, good practice)
                    # import os
                    # os.makedirs(output_directory, exist_ok=True)

                    with open(output_file_name, "w") as f:
                        json.dump(geojson_data_for_day, f, indent=2)
                    print(f"GeoJSON data for {current_date_in_loop.strftime('%Y-%m-%d')} saved to {output_file_name} ({len(geojson_features_for_day)} features)")

                except Exception as e:
                    print(f"Error during GeoJSON conversion or file writing for {current_date_in_loop.strftime('%Y-%m-%d')}: {e}")
                    # traceback.print_exc() # Uncomment for detailed traceback for this specific day

                # Move to the next day
                current_date_in_loop += timedelta(days=1)
            print("\nScript finished processing all dates.")
    else:
        print(f"Error: Item with ID '{item_id}' is of type '{item.type}', not a 'Feature Service'. Cannot process.")

except Exception as e:
    print(f"\nAn critical error occurred: {e}")
    print("Detailed traceback:")
    traceback.print_exc()

Script started. Processing from 2025-10-08 to 2025-10-10.
Attempting to query all features from layer: Calls For Service - Last 30 Days...
Successfully queried 64098 features in total.

Processing data for: 2025-10-08 (Year: 2025, DayOfYear: 281)
GeoJSON data for 2025-10-08 saved to F:/Data/Las_Vegas_Crime/Data/LVMPD/LVMPD_Calls_For_Service/FileTypes/GeoJSON/Daily/LVMPD_Calls_For_Service_20251008.geojson (2052 features)

Processing data for: 2025-10-09 (Year: 2025, DayOfYear: 282)
GeoJSON data for 2025-10-09 saved to F:/Data/Las_Vegas_Crime/Data/LVMPD/LVMPD_Calls_For_Service/FileTypes/GeoJSON/Daily/LVMPD_Calls_For_Service_20251009.geojson (2104 features)

Processing data for: 2025-10-10 (Year: 2025, DayOfYear: 283)
No features found for 2025-10-10.

Script finished processing all dates.


: 